# 箱体和其他零件的尺寸设计

我们现在已经得到了两级减速齿轮的所有参数，以及用工作扭矩计算的轴颈，我们把它封装到`design.design_all`里，可以通过输出牵引力、输送带速度、滚筒直径直接计算

In [1]:
from design import design_all
import numpy as np
import matplotlib.pyplot as plt
import json

F_w = 2.3 #v kN
v_w = 0.95 #m/s
D = 440 #mm

out1, out2, out_shaft = design_all(F_w, v_w, D)
z1_small,z2_big,m12,b1_small,b2_big,a12 = out1
z2_small,z3_big,m23,b2_small,b3_big,a23 = out2
d1,d2,d3 = out_shaft[0]

d1_small = z1_small * m12
d2_big = z2_big * m12
d2_small = z2_small * m23
d3_big = z3_big * m23


工作转速：41.236 rpm
总效率为: 0.808
电机所需功率为：2703.444 W
总传动比为:35.285
i1 = 6.773, i2 = 5.210
轴1的转速为:1455.000, 轴2的转速为:214.831, 轴3的转速为:41.236
P1 = 2119.887 W, P2 = 2035.939 W, P3 = 1955.316 W
d1 = 11.677 mm, d2 = 21.797 mm, d3 = 37.281 mm
T1 = 13.914 N*m, T2 = 90.505 N*m, T3 = 452.843 N*m
epsilon_a = 1.691
试算 d_1t = 32.64 mm
实际 d_1H = 34.47 mm
试算 m_t = 1.26 mm
K_Fb = 1.15
实际 m_F = 1.30 mm
实际 d_1F = 22.09 mm
齿面接触疲劳强度计算: m_H = 2.028, d_H = 34.469
齿根弯曲疲劳强度计算: m_F = 1.300, d_F = 22.094

    主要设计结论:
    齿数z1 = 17
    齿数z2 = 115
    模数m = 2 mm
    压力角alpha = 20°
    中心距a = 132 mm
    齿宽b1 = 32 mm
    齿宽b2 = 27 mm
    小齿轮选用40Cr(调质),
    大齿轮选用45钢(调质),
    7级精度
    
epsilon_a = 1.707
试算 d_1t = 59.57 mm
实际 d_1H = 62.04 mm
试算 m_t = 2.09 mm
K_Fb = 1.15
实际 m_F = 2.01 mm
实际 d_1F = 40.29 mm
齿面接触疲劳强度计算: m_H = 3.102, d_H = 62.043
齿根弯曲疲劳强度计算: m_F = 2.014, d_F = 40.288

    主要设计结论:
    齿数z1 = 31
    齿数z2 = 162
    模数m = 2 mm
    压力角alpha = 20°
    中心距a = 193 mm
    齿宽b1 = 54 mm
    齿宽b2 = 49 mm
    小齿轮选用40Cr(调质),
  

In [3]:
print('齿数: z1_small = {:.3f}, z2_big = {:.3f}, z2_small = {:.3f}, z3_big = {:.3f}'.format(z1_small,z2_big,z2_small,z3_big))
print('中心距 a12={:.3f} a23={:.3f}'.format(a12,a23))
print('齿宽 b1_small={:.3f} b2_big={:.3f} b2_small={:.3f} b3_big={:.3f}'.format(b1_small,b2_big,b2_small,b3_big))
print('轴颈下界: d1={:.3f} d2={:.3f} d3={:.3f}'.format(d1,d2,d3))
print('分度圆直径 d1_small={:.3f} d2_big={:.3f} d2_small={:.3f} d3_big={:.3f}'.format(d1_small,d2_big,d2_small,d3_big))

齿数: z1_small = 17.000, z2_big = 115.000, z2_small = 31.000, z3_big = 162.000
中心距 a12=132.000 a23=193.000
齿宽 b1_small=32.200 b2_big=27.200 b2_small=54.600 b3_big=49.600
轴颈下界: d1=11.677 d2=21.797 d3=37.281
分度圆直径 d1_small=34.000 d2_big=230.000 d2_small=62.000 d3_big=324.000


现在我们来设计箱体

## 设计螺栓

根据机械设计`课程设计 Page 23`我们发现每一个螺栓拥有三个参数

* $c_1$: 至外箱壁距离
* $c_2$: 至凸边缘的距离
* $D_0$: 沉头座坑直径
* $d$: 螺栓直径

因此，我们做了以下的事情
* 建立一个类`Bolt`用于表示螺栓
* 录制`课程设计 Page 23 表 5-3`为一个字典`Bolt_to_c1c2D0`
* 定义了一个函数`select_bolt(d)`，给定螺栓计算直径，选择合适的螺栓直径

In [9]:
ALL_BOLT = [8,10,12,16,20,24,30]
def select_bolt(d):
    k = ALL_BOLT
    k = np.array(k)
    d = np.abs(k - d)
    idx = np.argmin(d)
    return ALL_BOLT[idx]

Bolt_to_c1c2D0 = {
    8: (13,11,18),
    10: (16,14,22),
    12: (18,16,26),
    16: (22,20,33),
    20: (26,24,20),
    24: (34,28,48),
    30: (40,34,61)
}


class Bolt:
    def __init__(self, d):
        self.d = d
        self.d = select_bolt(self.d)
        self.c1, self.c2, self.D0 = Bolt_to_c1c2D0[self.d]
    def __dict__(self):
        return {'d':self.d, 'c1':self.c1, 'c2':self.c2, 'D0':self.D0}
    


## 设计箱体的其他尺寸

接下来，我们创建了一个字典`box`用于储存箱体的设计尺寸，根据`课程设计 Page 23 表5-2`进行设计各个尺寸。

In [5]:
box = {}

首先计算箱座壁厚，由`表5-1`知，对于二级圆柱齿轮减速器，由低速级中心距，即第二齿轮副的中心距 $a$ 决定

$$\delta = \max \{0.025a + 1, 8 \}$$

In [6]:
box['delta'] = max(8, 0.025*a23+3)
print('delta = {:.3f} mm'.format(box['delta']))

delta = 8.000 mm


箱盖壁厚：
$$ \delta_1 = \max \{ 0.85 \delta, 8 \} $$


In [7]:
box['delta_1'] = max(0.85*box['delta'], 8)
print('delta_1 = {:.3f} mm'.format(box['delta_1']))

delta_1 = 8.000 mm


地脚螺栓直径：
$$ d_f = 0.036a + 12 $$
传入`Bolt`中化整、选型

In [10]:
box['d_f'] = Bolt(0.036 * a23 + 12)
print('地脚螺栓d_f的各个参数为:')
print(box['d_f'].__dict__())

地脚螺栓d_f的各个参数为:
{'d': 20, 'c1': 26, 'c2': 24, 'D0': 20}


地脚螺栓个数
$$ n=\left\{\begin{matrix}
 4 & \text{if }a\le 250 \\
 6 & \text{if }a\gt 250
\end{matrix}\right. $$

In [12]:
box['n'] = 4 if a23<=250 else 6
print('地脚螺栓个数为: {}'.format(box['n']))

地脚螺栓个数为: 4


其余的参数，由`表5-2`设计得到

In [14]:
# 铸铁减速箱体结构之二
box['b'] = 1.5 * box['delta']
box['b_1'] = 1.5 * box['delta_1']
box['b_2'] = 2.5 * box['delta']
box['d_1'] = Bolt(0.75 * box['d_f'].d)
box['d_2'] = Bolt(0.6 * box['d_f'].d)
box['l'] = 200
box['d_3'] = Bolt(0.5 * box['d_f'].d)
box['d_4'] = Bolt(0.4 * box['d_f'].d)
box['d'] = 0.8 * box['d_2'].d
box['R_1'] = box['d_2'].c2
box['l_1'] = box['d_2'].c1 + box['d_2'].c2 + 8
box['m_1'] = 0.85 * box['delta_1']
box['m'] = 0.85 * box['delta']

box['Delta_1'] = box['delta']
box['Delta_2'] = box['delta']

为了方便表述，我们把轴向称为$x$向，称为前后，竖直方向称为$z$向，称为上下，$y=z\times x$称为左右。

接下来我们要设计分型面凸缘尺寸，如`Page 28 图6-1`所示：

* $A$: $y$方向内壁 到 凸缘$y$向外缘的距离
* $B$: $x$方向内壁 到 凸缘$x$向外缘的距离

他们的计算由`Page 29 行3`的文字得到

$$ A = \delta + c_1 + c_2 $$

$$ B = \delta + c_1 + c_2 + 5 $$

这里的$c_1,c_2$指的是`轴承旁连接螺栓$d_1$`的参数。

In [15]:
# 分箱面凸缘尺寸A
box['A'] = box['delta']+box['d_1'].c1+box['d_1'].c2+5
box['B'] = box['A'] + 5
print('分箱面凸缘尺寸 A={:.3f}, B={:.3f}'.format(box['A'], box['B']))

分箱面凸缘尺寸 A=55.000, B=60.000


接下来，我们计算轴、轴套、甩油环、轴承、轴承盖是如何叠在孔上的。

根据`Page 33 图6-9`，我们选取脂润滑，然后由图旁的文字得到

$$
\left\{\begin{matrix}
 \text{toOilSlinger} = 3 & 箱体内壁到甩油环 \\
\text{ToBearIn} = 12 & 箱体内壁到轴承内侧 \\
\text{Delta3} = 10 & 大齿轮距离 \\
\text{ShaftToBear} = 3 & 对于闭轴承端盖，端面到轴承外侧平面的距离
\end{matrix}\right.
$$



In [16]:
# 箱体内壁到甩油环
box['to_oil_slinger'] = 3
# 箱体内壁到轴承内侧
box['to_bear_in'] = 12
# 大齿轮距离
box['Delta_3'] = 10
# 闭轴承盖端, 轴面到轴承外面的距离
box['shaft_to_bear'] = 3

接下来我们计算浸油深度、中心高，首先根据`Page 30 图6-3`得到中心高

$$ \text{HCenterToBottom} = \text{d3Big}/2 + 30 $$

其次我们计算浸油深度，根据`Page 25 表5-4`表述
高速级大齿轮的浸油深度$h_f$为齿高的$0.7$ 与 $10$ 之间的较大值

$$ h_f = \max \{10, 0.7 m_{12}\times 2.25 \} $$

低速级大齿轮的浸油深度$h_s$为一个尺高和$\frac{1}{3}$个齿轮半径之间的较大值

$$ h_s = \max \{ 0.25m_{23}, \frac{1}{3}m_{23}z_{3,big}/2 \} $$

In [18]:
# 浸油深度
h_f = max(10, 0.7*m12*2.25)
h_s = [m23*2.25, 1/3*m23*z3_big/2]
h_expect = max(h_s)
box['h_s'] = h_expect
box['H_center_to_bottom'] = d3_big/2 + 30
box['delta_bottom'] = box['delta']
box['under_to_ground'] = 5
print('h_s = {:.3f} h_f = {:.3f} H_center_to_bottom = {:.3f} delta_bottom = {:.3f}'.format(h_expect, h_f, box['H_center_to_bottom'], box['delta_bottom']))

h_s = 54.000 h_f = 10.000 H_center_to_bottom = 192.000 delta_bottom = 8.000


## 其他零件

我们创建一个零件的基类`Part`来管理其他的零件，并存入一个注册表`ALL_PART`中，遍历它即可获得创建的所有零件，方便计算结果的保存。

In [19]:
ALL_PART = {}
class Part:
    def __init__(self,key_name):
        ALL_PART[key_name] = self

### 轴承

为了确定轴承，我们需要计算轴向载荷，思路是先获取每个轴传递的扭矩，然后根据齿轮直径和压力角计算径向力，最后简化为两端角度约束的弯曲梁问题计算两端的约束力，即可得到轴向载荷。

首先我们要获取每个转动副的转矩.


In [21]:
T1,T2,T3 = out_shaft[3]
print('三个轴传递的力矩分别为: T1={:.3f} N/mm, T2={:.3f} N/mm, T3={:.3f} N/mm'.format(T1,T2,T3))

三个轴传递的力矩分别为: T1=13.914 N/mm, T2=90.505 N/mm, T3=452.843 N/mm


接下来，我们先计算切向力大小
$$ F_t = \frac{T}{d/2} $$

然后用它和压力角$\alpha$一起计算径向力大小

$$ F_r = F_t \tan{(\alpha)} $$

In [ ]:
# 切向力
F1_small = T1 / d1_small * 2
F2_big = T2 / d2_big * 2
F2_small = T2 / d2_small * 2
F3_big = T3 / d3_big * 2
# 计算径向力
alpha = np.deg2rad(20)
F1_small_r = F1_small * np.tan(alpha)
F2_big_r = F2_big * np.tan(alpha)
F2_small_r = F2_small * np.tan(alpha)
F3_big_r = F3_big * np.tan(alpha)
# 输出结果
print('切向力为 F1_small = {:.3f} kN, F2_big = {:.3f} kN, F2_small = {:.3f} kN, F3_big = {:.3f} kN'.format(F1_small, F2_big, F2_small, F3_big))
print('径向力为 F1_small_r = {:.3f} kN, F2_big_r = {:.3f} kN, F2_small_r = {:.3f} kN, F3_big_r = {:.3f} kN'.format(F1_small_r, F2_big_r, F2_small_r, F3_big_r))

切向力为 F1_small = 0.818 kN, F2_big = 0.787 kN, F2_small = 2.920 kN, F3_big = 2.795 kN
径向力为 F1_small_r = 0.298 kN, F2_big_r = 0.286 kN, F2_small_r = 1.063 kN, F3_big_r = 1.017 kN


接下来，我们要确定受力点的轴向位置。

我们之前已经确定了：
* 箱体内侧到轴承内侧的距离`ToBearIn`
* 箱体内壁到齿轮的距离`Delta2`
* 各个齿轮的宽度`b`
* 大齿轮之间的距离`Delta3`

首先不难得到箱体内部$x$方向的宽度 `boxWidth`

$$ \text{boxWidth} = 2\Delta_2 + \Delta_3+ b_{2,big} + b_{3,big} + (b_{1,small}-b_{2,big})/2 + (b_{2,small}-b_{3,big})/2
 $$

In [24]:
box_width = 2*box['Delta_2'] + box['Delta_3'] + b2_big + b3_big + (b1_small-b2_big)/2 + (b2_small-b3_big)/2
print('减速箱内部宽: {:.3f}'.format(box_width))

减速箱内部宽: 107.800
